In [8]:
# LLM 모델을 통한 연산 예제
from typing import Type
from pydantic import BaseModel,Field
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for") # Field는 각 필드의 메타데이터(추가 설명 등)를 제공


# 주식 심볼 찾는 도구(종목 코드)
class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """

    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        # DuckDuckGo 검색 엔진을 통해 웹 검색을 수행하는 기능 또는 도구
        search_engine = DuckDuckGoSearchAPIWrapper()
        # 검색 엔진을 사용하여 웹 검색
        return search_engine.run(query)


agent = initialize_agent(
    llm=llm,
    verbose = True, 
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        # 종목 코드 검색 도구
        StockMarketSymbolSearchTool()
    ]
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare'}`


Read the latest posts from Cloudflare's official blog, covering topics such as security, performance, AI, email, DNS, and more. Learn how Cloudflare disrupts threats, innovates features, and improves Internet infrastructure. Finally, because Cloudflare also operates a CDN, websites that are already on Cloudflare will be given a "hot-path," and will load faster. We at Cloudflare are always striving to bring more privacy options to the open Internet, and we are excited to provide more private and secure browsing to Edge users. Cloudflare Announces Fourth Quarter and Fiscal Year 2023 Financial Results. February 08, 2024. New Cloudflare Report Shows Organizations Struggle to Identify and Manage Cybersecurity Risks of APIs. January 09, 2024. Cloudflare Announces Date of Fourth Quarter 2023 Financial Results. January 09, 2024. We list global press coverage about ... Cloudflare is

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "The stock market symbol for Cloudflare is NET. Now, let's analyze if Cloudflare's stock is a potential good investment.\n\nCloudflare is a company that provides services to speed up and secure websites and applications with its global network of servers. The company has shown growth in global traffic, indicating a growing dependency on Internet services.\n\nTo determine if Cloudflare's stock is a potential good investment, we need to consider factors such as the company's financial performance, growth potential, competitive landscape, and market trends. It's important to conduct a thorough analysis of the company's financial statements, revenue growth, profitability, and market position.\n\nAdditionally, you may want to consider consulting with a financial advisor or conducting further research on Cloudflare's stock pe